In [420]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras import backend as K
sess = K.get_session()
from tensorflow import keras
from tensorflow.keras import layers
import scipy
from scipy import stats


In [416]:
def KL(mu1, mu2, cov1, cov2):
    w2=np.log(np.abs(np.linalg.det(cov2)))
    w1=np.log(np.abs(np.linalg.det(cov1)))
    w2inv=np.linalg.inv(cov2)
    tr=np.trace(np.matmul(w2inv,cov1))
    mudiff=np.matmul(mu1-mu2,w2inv)
    mudiffcons=np.matmul(mudiff,mu1-mu2)
    xxxx=0.5*((w2-w1)+tr+mudiffcons)
    
    return(xxxx)

    

In [224]:
filed='./Downloads/IETR_DIBR_database_PNG/'
filelistd=os.listdir(filed)
dis_img = np.array([np.array(cv2.resize(cv2.cvtColor(cv2.imread(filed+fnamed),cv2.COLOR_BGR2GRAY),(100,100), interpolation = cv2.INTER_AREA)) for fnamed in filelistd])

In [225]:
fileo='./Downloads/Original_IETR/'
filelisto=os.listdir(fileo)
dis_orig = np.array([np.array(cv2.resize(cv2.cvtColor(cv2.imread(fileo+fnameo),cv2.COLOR_BGR2GRAY),(100,100), interpolation = cv2.INTER_AREA)) for fnameo in filelisto])

In [417]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [418]:
latent_dim = 20

encoder_inputs = keras.Input(shape=(100, 100, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 50, 50, 32)   320         input_16[0][0]                   
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 25, 25, 64)   18496       conv2d_42[0][0]                  
__________________________________________________________________________________________________
flatten_20 (Flatten)            (None, 40000)        0           conv2d_43[0][0]                  
____________________________________________________________________________________________

In [421]:
mean1,logvar1,samples1=encoder(dis_orig)
mu1= sess.run(mean1)
logsigma1=sess.run(logvar1)

print(mu1.shape)

(10, 20)


In [422]:
mean2,logvar2,samples2=encoder(dis_img)
mu2=sess.run(mean2)
logsigma2=sess.run(logvar2)
print(mu2.shape)

(140, 20)


In [423]:
print(logsigma2[4])

[  0.43582308  -4.117063     2.4031012   20.595675     1.0880939
   2.7125616  -13.419254    13.462277   -24.693962    -9.750143
 -11.461591     4.9881525    7.519195    -1.5786074  -10.187011
   0.60133     21.198849     9.406461    -8.372173    13.846533  ]


In [424]:
#making variance covariance matrix of both original and distorted image 
cov1=np.array([np.diag(logsigma1[i]) for i in range(len(dis_orig))])+0.00000001
cov2=np.array([np.diag(logsigma2[j]) for j in range(len(dis_img))])+0.00000001

In [425]:
KLD1=[]
KLD2=[]
KLD3=[]
KLD4=[]
KLD5=[]
KLD6=[]
KLD7=[]
KLD8=[]
KLD9=[]
KLD10=[]

for k1 in range(0,14):
    KLD1.append(KL(mu1[0],mu2[k1],cov1[0],cov2[k1]))
    
for k2 in range(14,28):
    KLD2.append(KL(mu1[1],mu2[k2],cov1[1],cov2[k2]))
    
for k3 in range(28,42):
    KLD3.append(KL(mu1[2],mu2[k3],cov1[2],cov2[k3]))
    
for k4 in range(42,56):
    KLD4.append(KL(mu1[3],mu2[k4],cov1[3],cov2[k4]))
    
for k5 in range(56,70):
    KLD5.append(KL(mu1[4],mu2[k5],cov1[4],cov2[k5]))
    
for k6 in range(70,84):
    KLD6.append(KL(mu1[5],mu2[k6],cov1[5],cov2[k6]))
    
for k7 in range(84,98):
    KLD7.append(KL(mu1[6],mu2[k7],cov1[6],cov2[k7]))
    
for k8 in range(98,112):
    KLD8.append(KL(mu1[7],mu2[k8],cov1[7],cov2[k8]))
    
for k9 in range(112,126):
    KLD9.append(KL(mu1[8],mu2[k9],cov1[8],cov2[k9]))
    
for k10 in range(126,140):
    KLD10.append(KL(mu1[9],mu2[k10],cov1[9],cov2[k10]))
    

In [426]:
KLD_syn=[]
KLD_syn.append(KLD3)
KLD_syn.append(KLD4)
KLD_syn.append(KLD10)

KLD_synf=np.array(KLD_syn)
KLD_syn=KLD_synf.flatten()
print(KLD_syn.shape)

(42,)


In [427]:
KLD_arr=[]
KLD_arr.append(KLD1)

KLD_arr.append(KLD4)
KLD_arr.append(KLD5)
KLD_arr.append(KLD6)
KLD_arr.append(KLD7)
KLD_arr.append(KLD8)
KLD_arr.append(KLD9)

KLD_arr=np.array(KLD_arr)

print(KLD_arr.shape)

(7, 14)


In [428]:
objective_scores_orig=KLD_arr.flatten()
objective_scores_syn=KLD_syn

""""mysocre=[]
mysocre.append(KLD1)
mysocre.append(KLD2)
mysocre.append(KLD3)
mysocre.append(KLD4)
mysocre.append(KLD5)
mysocre.append(KLD6)
mysocre.append(KLD7)
mysocre.append(KLD8)
mysocre.append(KLD9)
mysocre.append(KLD10)

score=np.array(mysocre)
score=score.flatten()"""



'"mysocre=[]\nmysocre.append(KLD1)\nmysocre.append(KLD2)\nmysocre.append(KLD3)\nmysocre.append(KLD4)\nmysocre.append(KLD5)\nmysocre.append(KLD6)\nmysocre.append(KLD7)\nmysocre.append(KLD8)\nmysocre.append(KLD9)\nmysocre.append(KLD10)\n\nscore=np.array(mysocre)\nscore=score.flatten()'

In [429]:
subjective_scores = np.array([0.591450046206532,
0.662523941863571,
0.482770611989651,
0.784196184203399,
0.282522277106338,
0.647464473052446,
0.305047110033448,
0.29277954439822,
0.489600552640186,
0.497514516912261,
0.499754554978394,
0.826163743067733,
0.162313593846142,
0.452732703190989,
0.679082366068258,
0.529551830265255,
0.672434449272598,
0.848435869440699,
0.239381573498838,
0.519363875165456,
0.098485509875757,
0.250368894907729,
0.774666840849666,
0.580021871381838,
0.682754501970252,
0.964006325086385,
0.432258795632313,
0.517068804044989,
0.863510062111633,
0.908324011242464,
0.738170293104672,
0.866538453029475,
0.81877185003214,
0.838498549081274,
0.036201738640763,
0.350598252172135,
0.619883844248706,
0.715318324601426,
0.673017796296823,
0.651627664348734,
0.505453784615257,
0.61210770043521,
0.240174830416431,
0.173654841694507,
0.530078954891447,
0.837074523344541,
0.478473785829511,
0.833064726381777,
0.369448429070808,
0.686376045932332,
0.560928889708332,
0.819290713651848,
0.416449954740804,
0.66758800796767,
0.149527360137351,
0.435486201211814,
0.032865927003549,
0.167095821472421,
0.633511207418824,
0.58086832685444,
0.498674311251136,
0.90645743512307,
0.389031972482008,
0.406285009058487,
0.595752358048127,
0.51135819227138,
0.518993287064144,
0.924073630989853,
0.330146860323064,
0.439550081679502,
0.282999409333829,
0.201693696265621,
0.775300901852862,
0.647753354344015,
0.551428333644776,
0.777188452399451,
0.367546271185079,
0.624584748177781,
0.754365815714084,
0.785579080784981,
0.77966129542017,
0.669638401087394,
0.133265501709099,
0.657534869077368,
0.090079728400751,
0.219564569389693,
0.583889735265927,
0.716360444074853,
0.594113188456475,
0.728014429567815,
0.107997663227676,
0.678428567551548,
0.710816964350032,
0.943633121341149,
0.705653821098252,
0.619209641653573,
0.264531428485334,
0.753319937363287,
0.036670518094472,
0.13962757767007,
0.745884386453196,
0.526282428013834,
0.660350709447516,
0.863305906634233,
0.2967184073528,
0.27047354346028,
0.719367700726814,
0.715783896474824,
0.590785267234655,
0.993188588327144,
0.499050685111133,
0.392189069256821,
0.047356890366184,
0.16789800389884,
0.628484045879569,
0.712783798798584,
0.723456269376853,
0.837042232299577,
0.255287851409832,
0.303467677517448,
0.690995710536917,
0.786814526447799,
0.734118125343431,
0.822787013990674,
0.280087692964191,
0.227792912603308,
0.02104915158573,
0.149706508121121,
0.693170061835007,
0.603005180641202,
0.581354154027684,
0.907311845453262,
0.206689847152443,
0.303516481939392,
0.714746037750429, 
0.628444329467981,
0.555000504270454,
0.9609795663716,
0.119566755981298,
0.275728490210608])

#Subjective scores of original images
new_sub_orig=np.array([0.591450046206532,
0.662523941863571,
0.482770611989651,
0.784196184203399,
0.282522277106338,
0.647464473052446,
0.305047110033448,
0.29277954439822,
0.489600552640186,
0.497514516912261,
0.499754554978394,
0.826163743067733,
0.162313593846142,
0.452732703190989,
0.240174830416431,
0.173654841694507,
0.530078954891447,
0.837074523344541,
0.478473785829511,
0.833064726381777,
0.369448429070808,
0.686376045932332,
0.560928889708332,
0.819290713651848,
0.416449954740804,
0.66758800796767,
0.149527360137351,
0.435486201211814,
0.032865927003549,
0.167095821472421,
0.633511207418824,
0.58086832685444,
0.498674311251136,
0.90645743512307,
0.389031972482008,
0.406285009058487,
0.595752358048127,
0.51135819227138,
0.518993287064144,
0.924073630989853,
0.330146860323064,
0.439550081679502,
0.282999409333829,
0.201693696265621,
0.775300901852862,
0.647753354344015,
0.551428333644776,
0.777188452399451,
0.367546271185079,
0.624584748177781,
0.754365815714084,
0.785579080784981,
0.77966129542017,
0.669638401087394,
0.133265501709099,
0.657534869077368,
0.090079728400751,
0.219564569389693,
0.583889735265927,
0.716360444074853,
0.594113188456475,
0.728014429567815,
0.107997663227676,
0.678428567551548,
0.710816964350032,
0.943633121341149,
0.705653821098252,
0.619209641653573,
0.264531428485334,
0.753319937363287,
0.036670518094472,
0.13962757767007,
0.745884386453196,
0.526282428013834,
0.660350709447516,
0.863305906634233,
0.2967184073528,
0.27047354346028,
0.719367700726814,
0.715783896474824,
0.590785267234655,
0.993188588327144,
0.499050685111133,
0.392189069256821,
0.047356890366184,
0.16789800389884,
0.628484045879569,
0.712783798798584,
0.723456269376853,
0.837042232299577,
0.255287851409832,
0.303467677517448,
0.690995710536917,
0.786814526447799,
0.734118125343431,
0.822787013990674,
0.280087692964191,
0.227792912603308
])

#sunjective scores of synthetic images

new_sub_syn=np.array([0.679082366068258,
0.529551830265255,
0.672434449272598,
0.848435869440699,
0.239381573498838,
0.519363875165456,
0.098485509875757,
0.250368894907729,
0.774666840849666,
0.580021871381838,
0.682754501970252,
0.964006325086385,
0.432258795632313,
0.517068804044989,
0.863510062111633,
0.908324011242464,
0.738170293104672,
0.866538453029475,
0.81877185003214,
0.838498549081274,
0.036201738640763,
0.350598252172135,
0.619883844248706,
0.715318324601426,
0.673017796296823,
0.651627664348734,
0.505453784615257,
0.61210770043521,
0.02104915158573,
0.149706508121121,
0.693170061835007,
0.603005180641202,
0.581354154027684,
0.907311845453262,
0.206689847152443,
0.303516481939392,
0.714746037750429, 
0.628444329467981,
0.555000504270454,
0.9609795663716,
0.119566755981298,
0.275728490210608])

In [430]:
corr_orig=scipy.stats.spearmanr(new_sub_orig,objective_scores_orig)
corr_syn=scipy.stats.spearmanr(new_sub_syn,objective_scores_syn)


In [431]:
#best result from VAE corr coeff
final=np.power(np.abs(corr_orig[0]),0.7)+np.power(np.abs(corr_syn[0]),0.3)
print(final)

0.8384068459384877


0.03559948393868492
